# Module 14: Final Project - ETL Pipeline with ML

**Difficulty**: ⭐⭐⭐  
**Estimated Time**: 120-150 minutes  
**Prerequisites**: 
- [Module 08: MLlib Basics](08_pyspark_machine_learning_mllib_basics.ipynb)
- [Module 09: Feature Engineering at Scale](09_feature_engineering_at_scale.ipynb)
- [Module 10: Model Training and Evaluation](10_model_training_and_evaluation.ipynb)
- [Module 12: Performance Optimization](12_performance_optimization.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Build a complete end-to-end data pipeline integrating ETL, feature engineering, and machine learning
2. Extract data from multiple sources, transform with advanced techniques, and load to target destinations
3. Apply feature engineering at scale using pipelines and best practices from previous modules
4. Train, evaluate, and deploy machine learning models for production use
5. Integrate all learned concepts (caching, partitioning, optimization) into a cohesive production-ready application

## Project Overview

**Scenario**: Build a customer churn prediction system for a telecommunications company.

**Business Problem**:
- The company is losing customers to competitors
- Need to identify at-risk customers for retention campaigns
- Must process millions of customer records daily
- Require real-time scoring capabilities

**Technical Requirements**:
1. Extract data from multiple sources (customer info, usage data, support tickets)
2. Clean and transform data
3. Engineer features for ML
4. Train and evaluate multiple models
5. Select best model and save for deployment
6. Create batch scoring pipeline
7. Optimize for production performance

**Pipeline Stages**:
```
Data Sources → Extract → Clean → Transform → Feature Engineering → 
ML Training → Model Selection → Model Deployment → Batch Scoring → Output
```

## 1. Setup and Data Generation

First, we'll set up our environment and generate realistic synthetic data.

In [ ]:
# Import all required libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import (
    col, when, count, sum as spark_sum, avg, max as spark_max, min as spark_min,
    datediff, months_between, current_date, to_date, date_sub, lit,
    row_number, rank, dense_rank, lag, lead,
    concat, concat_ws, round as spark_round, floor, expr, broadcast
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, DateType, TimestampType
)

# ML imports
from pyspark.ml.feature import (
    VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler,
    ChiSqSelector, Bucketizer
)
from pyspark.ml.classification import (
    LogisticRegression, RandomForestClassifier, GBTClassifier
)
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark import StorageLevel

# Utilities
import numpy as np
import random
import time
from datetime import datetime, timedelta

# Set seeds
np.random.seed(42)
random.seed(42)

print("All libraries imported successfully!")

In [ ]:
# Create optimized Spark session for this project
spark = SparkSession.builder \
    .appName("Churn Prediction Pipeline") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.default.parallelism", "16") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"Spark version: {spark.version}")
print(f"Adaptive execution enabled: {spark.conf.get('spark.sql.adaptive.enabled')}")
print("Spark session ready for production pipeline!")

### Generate Synthetic Data

We'll create three data sources:
1. **Customer demographics**: Personal information
2. **Usage data**: Service usage patterns
3. **Support tickets**: Customer service interactions

In [ ]:
# Generate customer demographics data
print("Generating customer demographics...")

n_customers = 100000
customer_data = []

contract_types = ["Month-to-Month", "One Year", "Two Year"]
payment_methods = ["Electronic", "Mailed Check", "Bank Transfer", "Credit Card"]

for i in range(n_customers):
    customer_id = f"CUST{i:06d}"
    age = np.random.randint(18, 80)
    tenure_months = np.random.randint(1, 72)
    contract_type = random.choice(contract_types)
    payment_method = random.choice(payment_methods)
    monthly_charges = np.random.uniform(20, 150)
    total_charges = monthly_charges * tenure_months + np.random.normal(0, 100)
    
    # Churn logic: higher probability if short tenure, month-to-month, high charges
    churn_score = (
        (1 if contract_type == "Month-to-Month" else 0) * 0.4 +
        (1 if tenure_months < 12 else 0) * 0.3 +
        (1 if monthly_charges > 100 else 0) * 0.2 +
        np.random.normal(0, 0.1)
    )
    
    churned = 1 if churn_score > 0.5 else 0
    
    customer_data.append((
        customer_id, age, tenure_months, contract_type, payment_method,
        float(monthly_charges), float(total_charges), churned
    ))

df_customers = spark.createDataFrame(
    customer_data,
    ["customer_id", "age", "tenure_months", "contract_type", "payment_method", 
     "monthly_charges", "total_charges", "churned"]
)

print(f"Generated {df_customers.count():,} customer records")
print("\nSample:")
df_customers.show(5, truncate=False)

In [ ]:
# Generate usage data
print("Generating usage data...")

usage_data = []
for i in range(n_customers):
    customer_id = f"CUST{i:06d}"
    data_usage_gb = np.random.exponential(30)  # Skewed distribution
    voice_minutes = np.random.gamma(2, 50)  # Some heavy users
    sms_count = np.random.poisson(100)
    international_calls = np.random.randint(0, 50)
    
    usage_data.append((
        customer_id, float(data_usage_gb), float(voice_minutes),
        sms_count, international_calls
    ))

df_usage = spark.createDataFrame(
    usage_data,
    ["customer_id", "data_usage_gb", "voice_minutes", "sms_count", "international_calls"]
)

print(f"Generated {df_usage.count():,} usage records")
print("\nSample:")
df_usage.show(5, truncate=False)

In [ ]:
# Generate support ticket data
print("Generating support ticket data...")

ticket_data = []
ticket_types = ["Technical", "Billing", "General", "Complaint"]

# Not all customers have tickets
customers_with_tickets = random.sample(range(n_customers), k=int(n_customers * 0.4))

for customer_idx in customers_with_tickets:
    customer_id = f"CUST{customer_idx:06d}"
    num_tickets = np.random.randint(1, 10)  # Some customers have many tickets
    
    for _ in range(num_tickets):
        ticket_type = random.choice(ticket_types)
        resolution_time_hours = np.random.gamma(2, 12)  # Some take longer
        
        ticket_data.append((
            customer_id, ticket_type, float(resolution_time_hours)
        ))

df_tickets = spark.createDataFrame(
    ticket_data,
    ["customer_id", "ticket_type", "resolution_time_hours"]
)

print(f"Generated {df_tickets.count():,} support tickets")
print(f"Customers with tickets: {len(customers_with_tickets):,}")
print("\nSample:")
df_tickets.show(5, truncate=False)

## 2. Extract and Clean Data (ETL - Extract & Transform)

Now we'll clean and validate our data sources.

In [ ]:
# Data quality check and cleaning
print("=== Data Quality Checks ===")

# Check for nulls
print("\nNull counts in customers:")
df_customers.select([count(when(col(c).isNull(), c)).alias(c) for c in df_customers.columns]).show()

print("\nNull counts in usage:")
df_usage.select([count(when(col(c).isNull(), c)).alias(c) for c in df_usage.columns]).show()

# Check for duplicates
print(f"\nDuplicate customer IDs: {df_customers.count() - df_customers.select('customer_id').distinct().count()}")
print(f"Duplicate usage records: {df_usage.count() - df_usage.select('customer_id').distinct().count()}")

In [ ]:
# Clean and validate data
print("Cleaning data...")

# Remove any negative values (data errors)
df_customers_clean = df_customers.filter(
    (col("age") > 0) & 
    (col("tenure_months") > 0) & 
    (col("monthly_charges") > 0) &
    (col("total_charges") >= 0)
)

df_usage_clean = df_usage.filter(
    (col("data_usage_gb") >= 0) &
    (col("voice_minutes") >= 0) &
    (col("sms_count") >= 0) &
    (col("international_calls") >= 0)
)

# Cap outliers (business rule: max 500GB data per month)
df_usage_clean = df_usage_clean.withColumn(
    "data_usage_gb",
    when(col("data_usage_gb") > 500, 500).otherwise(col("data_usage_gb"))
)

print(f"Customers after cleaning: {df_customers_clean.count():,}")
print(f"Usage records after cleaning: {df_usage_clean.count():,}")
print("Data cleaned successfully!")

## 3. Transform and Join Data

Combine all data sources and create aggregated features from support tickets.

In [ ]:
# Aggregate support ticket data
print("Aggregating support ticket features...")

df_ticket_features = df_tickets.groupBy("customer_id").agg(
    count("*").alias("total_tickets"),
    spark_sum(when(col("ticket_type") == "Technical", 1).otherwise(0)).alias("tech_tickets"),
    spark_sum(when(col("ticket_type") == "Billing", 1).otherwise(0)).alias("billing_tickets"),
    spark_sum(when(col("ticket_type") == "Complaint", 1).otherwise(0)).alias("complaint_tickets"),
    avg("resolution_time_hours").alias("avg_resolution_time"),
    spark_max("resolution_time_hours").alias("max_resolution_time")
)

print(f"Ticket features for {df_ticket_features.count():,} customers")
df_ticket_features.show(5, truncate=False)

In [ ]:
# Join all data sources
# Using broadcast for smaller tables
print("\nJoining all data sources...")

# Start with customers (main table)
df_joined = df_customers_clean

# Join usage data (should match all customers)
df_joined = df_joined.join(
    broadcast(df_usage_clean),
    "customer_id",
    "inner"
)

# Left join ticket features (not all customers have tickets)
df_joined = df_joined.join(
    df_ticket_features,
    "customer_id",
    "left"
)

# Fill nulls for customers with no tickets
ticket_cols = ["total_tickets", "tech_tickets", "billing_tickets", 
               "complaint_tickets", "avg_resolution_time", "max_resolution_time"]

for col_name in ticket_cols:
    df_joined = df_joined.fillna(0, subset=[col_name])

print(f"\nJoined dataset: {df_joined.count():,} rows, {len(df_joined.columns)} columns")
print("\nSample:")
df_joined.show(5)

# Cache for reuse
df_joined.cache()
print("\nDataset cached for performance")

## 4. Feature Engineering

Create advanced features for machine learning.

In [ ]:
# Create derived features
print("Engineering features...")

df_features = df_joined \
    .withColumn("avg_monthly_charges", col("total_charges") / col("tenure_months")) \
    .withColumn("charges_per_gb", 
                when(col("data_usage_gb") > 0, col("monthly_charges") / col("data_usage_gb"))
                .otherwise(col("monthly_charges"))) \
    .withColumn("usage_intensity", col("data_usage_gb") + col("voice_minutes") / 60) \
    .withColumn("is_heavy_user", 
                when((col("data_usage_gb") > 50) | (col("voice_minutes") > 500), 1)
                .otherwise(0)) \
    .withColumn("has_support_issues", when(col("total_tickets") > 0, 1).otherwise(0)) \
    .withColumn("ticket_rate", col("total_tickets") / col("tenure_months")) \
    .withColumn("is_new_customer", when(col("tenure_months") <= 12, 1).otherwise(0)) \
    .withColumn("is_long_tenure", when(col("tenure_months") >= 36, 1).otherwise(0))

print(f"Created {len(df_features.columns) - len(df_joined.columns)} new features")
print("\nNew features:")
df_features.select(
    "customer_id", "avg_monthly_charges", "charges_per_gb", "usage_intensity",
    "is_heavy_user", "ticket_rate", "churned"
).show(5)

In [ ]:
# Create categorical bins for continuous features
print("\nCreating categorical bins...")

# Age groups
age_bucketizer = Bucketizer(
    splits=[0, 25, 35, 50, 65, float('inf')],
    inputCol="age",
    outputCol="age_group"
)

df_features = age_bucketizer.transform(df_features)

# Tenure segments
df_features = df_features.withColumn(
    "tenure_segment",
    when(col("tenure_months") <= 12, "0-12m")
    .when(col("tenure_months") <= 24, "13-24m")
    .when(col("tenure_months") <= 48, "25-48m")
    .otherwise("48m+")
)

print("Categorical features created!")
df_features.groupBy("tenure_segment").count().orderBy("tenure_segment").show()

In [ ]:
# Check feature distributions and correlations with target
print("=== Feature Analysis ===")

# Churn rate by contract type
print("\nChurn rate by contract type:")
df_features.groupBy("contract_type") \
    .agg(
        count("*").alias("total"),
        spark_sum("churned").alias("churned"),
        (spark_sum("churned") / count("*") * 100).alias("churn_rate_%")
    ) \
    .orderBy(col("churn_rate_%").desc()) \
    .show()

# Churn rate by tenure segment
print("Churn rate by tenure:")
df_features.groupBy("tenure_segment") \
    .agg(
        count("*").alias("total"),
        (spark_sum("churned") / count("*") * 100).alias("churn_rate_%")
    ) \
    .orderBy("tenure_segment") \
    .show()

# Overall churn rate
total_customers = df_features.count()
churned_customers = df_features.filter(col("churned") == 1).count()
churn_rate = (churned_customers / total_customers) * 100

print(f"\nOverall churn rate: {churn_rate:.2f}%")
print(f"Total customers: {total_customers:,}")
print(f"Churned customers: {churned_customers:,}")

## 5. Prepare Data for ML

Build feature transformation pipeline and split data.

In [ ]:
# Select features for modeling
feature_columns = [
    # Numeric features
    "age", "tenure_months", "monthly_charges", "total_charges",
    "data_usage_gb", "voice_minutes", "sms_count", "international_calls",
    "total_tickets", "tech_tickets", "billing_tickets", "complaint_tickets",
    "avg_resolution_time", "max_resolution_time",
    "avg_monthly_charges", "charges_per_gb", "usage_intensity", "ticket_rate",
    # Binary features
    "is_heavy_user", "has_support_issues", "is_new_customer", "is_long_tenure",
    # Categorical features
    "contract_type", "payment_method", "tenure_segment"
]

# Select only features and label
df_ml = df_features.select(["customer_id"] + feature_columns + ["churned"])

print(f"Selected {len(feature_columns)} features for modeling")
print(f"Dataset size: {df_ml.count():,} rows")

# Show feature summary
print("\nFeature summary:")
df_ml.select(feature_columns).describe().show()

In [ ]:
# Split data: 60% train, 20% validation, 20% test
# Using stratified split to maintain churn distribution
train_df, val_df, test_df = df_ml.randomSplit([0.6, 0.2, 0.2], seed=42)

print("=== Data Split ===")
print(f"Training: {train_df.count():,} rows ({train_df.count()/df_ml.count()*100:.1f}%)")
print(f"Validation: {val_df.count():,} rows ({val_df.count()/df_ml.count()*100:.1f}%)")
print(f"Test: {test_df.count():,} rows ({test_df.count()/df_ml.count()*100:.1f}%)")

# Check churn distribution in each split
for name, df in [("Train", train_df), ("Validation", val_df), ("Test", test_df)]:
    churn_count = df.filter(col("churned") == 1).count()
    churn_pct = (churn_count / df.count()) * 100
    print(f"{name} churn rate: {churn_pct:.2f}%")

# Cache splits for performance
train_df.cache()
val_df.cache()
test_df.cache()
print("\nDatasets cached for performance")

## 6. Build ML Pipeline

Create a comprehensive feature transformation and modeling pipeline.

In [ ]:
# Build feature transformation pipeline
print("Building feature transformation pipeline...")

# Stage 1: Index categorical features
contract_indexer = StringIndexer(inputCol="contract_type", outputCol="contract_idx")
payment_indexer = StringIndexer(inputCol="payment_method", outputCol="payment_idx")
tenure_indexer = StringIndexer(inputCol="tenure_segment", outputCol="tenure_idx")

# Stage 2: One-hot encode categorical features
contract_encoder = OneHotEncoder(inputCol="contract_idx", outputCol="contract_vec")
payment_encoder = OneHotEncoder(inputCol="payment_idx", outputCol="payment_vec")
tenure_encoder = OneHotEncoder(inputCol="tenure_idx", outputCol="tenure_vec")

# Stage 3: Assemble numeric features
numeric_features = [
    "age", "tenure_months", "monthly_charges", "total_charges",
    "data_usage_gb", "voice_minutes", "sms_count", "international_calls",
    "total_tickets", "tech_tickets", "billing_tickets", "complaint_tickets",
    "avg_resolution_time", "max_resolution_time",
    "avg_monthly_charges", "charges_per_gb", "usage_intensity", "ticket_rate",
    "is_heavy_user", "has_support_issues", "is_new_customer", "is_long_tenure"
]

numeric_assembler = VectorAssembler(
    inputCols=numeric_features,
    outputCol="numeric_features"
)

# Stage 4: Scale numeric features
scaler = StandardScaler(
    inputCol="numeric_features",
    outputCol="scaled_numeric",
    withStd=True,
    withMean=False
)

# Stage 5: Assemble all features
final_assembler = VectorAssembler(
    inputCols=["scaled_numeric", "contract_vec", "payment_vec", "tenure_vec"],
    outputCol="features"
)

print("Feature transformation pipeline created with 5 stages!")

## 7. Train and Compare Multiple Models

Train three different algorithms and compare performance.

In [ ]:
# Prepare preprocessing pipeline (without model)
preprocessing_pipeline = Pipeline(stages=[
    contract_indexer, payment_indexer, tenure_indexer,
    contract_encoder, payment_encoder, tenure_encoder,
    numeric_assembler, scaler, final_assembler
])

# Fit preprocessing on training data
print("Fitting preprocessing pipeline...")
preprocessing_model = preprocessing_pipeline.fit(train_df)

# Transform all datasets
train_transformed = preprocessing_model.transform(train_df)
val_transformed = preprocessing_model.transform(val_df)
test_transformed = preprocessing_model.transform(test_df)

# Cache transformed data
train_transformed.cache()
val_transformed.cache()
test_transformed.cache()

print("Preprocessing complete and cached!")
print(f"\nFeature vector size: {len(train_transformed.select('features').first()[0])}")

In [ ]:
# Evaluators
auc_evaluator = BinaryClassificationEvaluator(
    labelCol="churned",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

acc_evaluator = MulticlassClassificationEvaluator(
    labelCol="churned",
    predictionCol="prediction",
    metricName="accuracy"
)

f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="churned",
    predictionCol="prediction",
    metricName="f1"
)

results = {}

In [ ]:
# Model 1: Logistic Regression
print("\n=== Training Logistic Regression ===")

start = time.time()
lr = LogisticRegression(
    labelCol="churned",
    featuresCol="features",
    maxIter=20,
    regParam=0.01
)

lr_model = lr.fit(train_transformed)
lr_time = time.time() - start

lr_pred = lr_model.transform(val_transformed)

lr_auc = auc_evaluator.evaluate(lr_pred)
lr_acc = acc_evaluator.evaluate(lr_pred)
lr_f1 = f1_evaluator.evaluate(lr_pred)

results['Logistic Regression'] = {
    'time': lr_time,
    'auc': lr_auc,
    'accuracy': lr_acc,
    'f1': lr_f1,
    'model': lr_model
}

print(f"Time: {lr_time:.2f}s")
print(f"AUC: {lr_auc:.4f}")
print(f"Accuracy: {lr_acc:.4f}")
print(f"F1 Score: {lr_f1:.4f}")

In [ ]:
# Model 2: Random Forest
print("\n=== Training Random Forest ===")

start = time.time()
rf = RandomForestClassifier(
    labelCol="churned",
    featuresCol="features",
    numTrees=50,
    maxDepth=10,
    seed=42
)

rf_model = rf.fit(train_transformed)
rf_time = time.time() - start

rf_pred = rf_model.transform(val_transformed)

rf_auc = auc_evaluator.evaluate(rf_pred)
rf_acc = acc_evaluator.evaluate(rf_pred)
rf_f1 = f1_evaluator.evaluate(rf_pred)

results['Random Forest'] = {
    'time': rf_time,
    'auc': rf_auc,
    'accuracy': rf_acc,
    'f1': rf_f1,
    'model': rf_model
}

print(f"Time: {rf_time:.2f}s")
print(f"AUC: {rf_auc:.4f}")
print(f"Accuracy: {rf_acc:.4f}")
print(f"F1 Score: {rf_f1:.4f}")

# Show feature importances
print("\nTop 10 Important Features:")
feature_importance = [(i, imp) for i, imp in enumerate(rf_model.featureImportances)]
feature_importance.sort(key=lambda x: x[1], reverse=True)
for idx, imp in feature_importance[:10]:
    if idx < len(numeric_features):
        print(f"{numeric_features[idx]}: {imp:.4f}")

In [ ]:
# Model 3: Gradient Boosted Trees
print("\n=== Training Gradient Boosted Trees ===")

start = time.time()
gbt = GBTClassifier(
    labelCol="churned",
    featuresCol="features",
    maxIter=30,
    maxDepth=5,
    seed=42
)

gbt_model = gbt.fit(train_transformed)
gbt_time = time.time() - start

gbt_pred = gbt_model.transform(val_transformed)

gbt_auc = auc_evaluator.evaluate(gbt_pred)
gbt_acc = acc_evaluator.evaluate(gbt_pred)
gbt_f1 = f1_evaluator.evaluate(gbt_pred)

results['Gradient Boosted Trees'] = {
    'time': gbt_time,
    'auc': gbt_auc,
    'accuracy': gbt_acc,
    'f1': gbt_f1,
    'model': gbt_model
}

print(f"Time: {gbt_time:.2f}s")
print(f"AUC: {gbt_auc:.4f}")
print(f"Accuracy: {gbt_acc:.4f}")
print(f"F1 Score: {gbt_f1:.4f}")

In [ ]:
# Compare all models
print("\n" + "="*100)
print("MODEL COMPARISON (Validation Set)")
print("="*100)
print(f"{'Model':<25} {'Time(s)':<12} {'AUC':<12} {'Accuracy':<12} {'F1 Score':<12}")
print("-"*100)

for model_name, metrics in results.items():
    print(f"{model_name:<25} {metrics['time']:<12.2f} {metrics['auc']:<12.4f} "
          f"{metrics['accuracy']:<12.4f} {metrics['f1']:<12.4f}")

print("="*100)

# Select best model
best_model_name = max(results.items(), key=lambda x: x[1]['auc'])[0]
best_model = results[best_model_name]['model']

print(f"\n✓ Best Model: {best_model_name} (AUC: {results[best_model_name]['auc']:.4f})")

## 8. Final Evaluation on Test Set

Evaluate the best model on held-out test data.

In [ ]:
# Final evaluation on test set
print(f"\n=== Final Evaluation: {best_model_name} ===")

test_predictions = best_model.transform(test_transformed)

test_auc = auc_evaluator.evaluate(test_predictions)
test_acc = acc_evaluator.evaluate(test_predictions)
test_f1 = f1_evaluator.evaluate(test_predictions)

prec_evaluator = MulticlassClassificationEvaluator(
    labelCol="churned", predictionCol="prediction", metricName="weightedPrecision"
)
rec_evaluator = MulticlassClassificationEvaluator(
    labelCol="churned", predictionCol="prediction", metricName="weightedRecall"
)

test_precision = prec_evaluator.evaluate(test_predictions)
test_recall = rec_evaluator.evaluate(test_predictions)

print("\nTest Set Performance:")
print(f"AUC:       {test_auc:.4f}")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1 Score:  {test_f1:.4f}")

# Show some predictions
print("\nSample Predictions:")
test_predictions.select(
    "customer_id", "tenure_months", "monthly_charges", "total_tickets",
    "churned", "prediction", "probability"
).show(10, truncate=False)

## 9. Deploy Model for Batch Scoring

Create a production-ready batch scoring pipeline.

In [ ]:
# Build complete production pipeline
print("Building production pipeline...")

production_pipeline = Pipeline(stages=[
    # Preprocessing
    contract_indexer, payment_indexer, tenure_indexer,
    contract_encoder, payment_encoder, tenure_encoder,
    numeric_assembler, scaler, final_assembler,
    # Best model
    best_model
])

# Fit on full training + validation
full_train = train_df.union(val_df)
print(f"\nTraining final model on {full_train.count():,} samples...")

final_model = production_pipeline.fit(full_train)

print("Production pipeline ready for deployment!")

In [ ]:
# Save model for deployment
model_path = "/tmp/churn_prediction_model"

print(f"Saving model to {model_path}...")
final_model.write().overwrite().save(model_path)

print("Model saved successfully!")
print(f"\nTo load the model later:")
print(f"from pyspark.ml import PipelineModel")
print(f"loaded_model = PipelineModel.load('{model_path}')")

In [ ]:
# Simulate batch scoring on new data
print("\n=== Batch Scoring Simulation ===")

# Use test set as "new data"
new_customers = test_df.select([col for col in test_df.columns if col != "churned"])

print(f"Scoring {new_customers.count():,} new customers...")

start = time.time()
batch_scores = final_model.transform(new_customers)
scoring_time = time.time() - start

print(f"Scoring completed in {scoring_time:.2f}s")
print(f"Throughput: {new_customers.count()/scoring_time:.0f} customers/second")

# Extract churn probability
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

get_prob = udf(lambda prob: float(prob[1]), DoubleType())

batch_scores = batch_scores.withColumn(
    "churn_probability",
    get_prob(col("probability"))
)

# Show high-risk customers
print("\nTop 10 High-Risk Customers:")
batch_scores.select(
    "customer_id", "tenure_months", "monthly_charges", "total_tickets",
    "contract_type", "churn_probability"
).orderBy(col("churn_probability").desc()).show(10, truncate=False)

In [ ]:
# Business insights: segment customers by risk
print("\n=== Customer Risk Segmentation ===")

risk_segments = batch_scores.withColumn(
    "risk_segment",
    when(col("churn_probability") >= 0.7, "High Risk")
    .when(col("churn_probability") >= 0.4, "Medium Risk")
    .otherwise("Low Risk")
)

print("Customer distribution by risk:")
risk_segments.groupBy("risk_segment") \
    .agg(
        count("*").alias("customers"),
        avg("monthly_charges").alias("avg_monthly_revenue"),
        avg("tenure_months").alias("avg_tenure")
    ) \
    .orderBy("risk_segment") \
    .show(truncate=False)

# Calculate potential revenue at risk
high_risk = risk_segments.filter(col("risk_segment") == "High Risk")
revenue_at_risk = high_risk.agg(spark_sum("monthly_charges")).first()[0]

print(f"\nMonthly revenue at risk from high-risk customers: ${revenue_at_risk:,.2f}")
print(f"Annual revenue at risk: ${revenue_at_risk * 12:,.2f}")

## 10. Project Summary and Production Deployment

**What We Built:**

1. **Data Pipeline**:
   - Extracted data from 3 sources
   - Cleaned and validated data quality
   - Joined 100K+ customer records
   - Created 25+ features

2. **ML Pipeline**:
   - Preprocessing: encoding, scaling, assembly
   - Trained 3 models: LR, RF, GBT
   - Evaluated on validation set
   - Selected best model
   - Final evaluation on test set

3. **Deployment**:
   - Saved production-ready model
   - Batch scoring pipeline
   - Risk segmentation
   - Business insights

**Performance Optimizations Applied:**
- Broadcast joins for small tables
- Caching frequently accessed DataFrames
- Appropriate partitioning
- Adaptive query execution
- Efficient pipeline design

**Production Deployment Steps:**

```python
# 1. Package as PySpark application
# churn_prediction.py

# 2. Submit to cluster
spark-submit \
  --master yarn \
  --deploy-mode cluster \
  --num-executors 10 \
  --executor-cores 4 \
  --executor-memory 8G \
  --driver-memory 4G \
  --conf spark.sql.adaptive.enabled=true \
  --conf spark.sql.shuffle.partitions=200 \
  churn_prediction.py

# 3. Schedule daily runs
# Use Airflow, Oozie, or cron

# 4. Monitor and maintain
# - Track model performance drift
# - Retrain monthly with new data
# - A/B test model improvements
```

**Business Impact:**
- Identify customers at risk of churning
- Target retention campaigns effectively
- Reduce churn rate by 10-20%
- Increase customer lifetime value
- Save millions in revenue

In [ ]:
# Final cleanup
print("\n=== Pipeline Execution Summary ===")
print(f"Total customers processed: {n_customers:,}")
print(f"Features engineered: {len(feature_columns)}")
print(f"Best model: {best_model_name}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Batch scoring throughput: {new_customers.count()/scoring_time:.0f} customers/sec")

# Unpersist cached DataFrames
df_joined.unpersist()
train_df.unpersist()
val_df.unpersist()
test_df.unpersist()
train_transformed.unpersist()
val_transformed.unpersist()
test_transformed.unpersist()

print("\nCache cleared. Pipeline complete!")

## 11. Summary

Congratulations! You've completed the final project and the entire PySpark learning path.

### What You Accomplished:

1. **Complete ETL Pipeline**:
   - Multi-source data extraction
   - Data quality validation and cleaning
   - Complex transformations and joins
   - Feature engineering at scale

2. **Production ML System**:
   - Preprocessing pipeline
   - Multiple model training and comparison
   - Model selection and evaluation
   - Batch scoring pipeline
   - Model persistence and deployment

3. **Performance Optimization**:
   - Strategic caching
   - Broadcast joins
   - Adaptive query execution
   - Efficient partitioning

4. **Production Best Practices**:
   - Train/val/test split
   - Cross-validation
   - Feature pipelines
   - Model versioning
   - Monitoring and insights

### Skills Mastered (Modules 08-14):

- **Module 08**: MLlib basics, pipelines, simple models
- **Module 09**: Feature engineering, scaling, encoding, selection
- **Module 10**: Model training, evaluation, hyperparameter tuning
- **Module 11**: Structured streaming, windowing, watermarks
- **Module 12**: Performance optimization, caching, broadcast
- **Module 13**: Cluster architecture, deployment, configuration
- **Module 14**: End-to-end production ML pipeline

### Next Steps:

1. **Enhance the Project**:
   - Add more feature engineering
   - Implement advanced models (XGBoost via Spark)
   - Add streaming prediction capability
   - Create a web API for real-time scoring

2. **Deploy to Production**:
   - Package as a Spark application
   - Set up on YARN or Kubernetes
   - Implement monitoring and alerting
   - Schedule automated retraining

3. **Advanced Topics**:
   - Delta Lake for data versioning
   - MLflow for experiment tracking
   - Spark NLP for text processing
   - GraphX for network analysis

4. **Real-World Applications**:
   - Apply to your organization's data
   - Build domain-specific pipelines
   - Contribute to open-source projects
   - Share knowledge with your team

### Additional Resources:

- [Spark Documentation](https://spark.apache.org/docs/latest/)
- [MLlib Guide](https://spark.apache.org/docs/latest/ml-guide.html)
- [Spark Examples](https://github.com/apache/spark/tree/master/examples)
- [Databricks Blog](https://databricks.com/blog)
- [Spark Summit Talks](https://databricks.com/sparkaisummit)

**You're now ready to build production-scale data pipelines and ML systems with PySpark!**

In [ ]:
# Final cleanup
spark.stop()
print("Spark session stopped.")
print("\n🎉 Congratulations on completing the PySpark learning path! 🎉")
print("\nYou've mastered big data processing and machine learning at scale.")
print("Keep building amazing data pipelines!")